# Search Engine


Zweite Belegaufgabe aus dem Fach Contentmanagement / Text- und Suchtechnologien (WT-2) an der HTW Berlin bei Herrn Prof. Dr. Zhang.


- *Sebastian Schmid s0543196)*

In [1]:
from subprocess import check_output
import json
from pprint import pprint
import numpy as np
import pandas as pd
from math import log
import re 
!ls

 graded02.pdf	  my_spider.py		 stop_words.txt
 index.txt	  pageranke_search.txt	 tfidf_search.txt
 my_spider.json   __pycache__		'WT-2 zweite Belegaufgabe.ipynb'
 my_spider.log	  rank.txt


In [2]:
from IPython.display import IFrame
#IFrame("./graded02.pdf", width=900, height=1400)

## a ) Der Crawler

### Code

In [3]:
%%writefile ./my_spider.py
# %%writefile ./my_spider.py
# %load ./my_spider.py

import scrapy
import json

class HtwSpider(scrapy.Spider):
    name = 'htw_spider'
    
    start_urls = [
        'http://people.f4.htw-berlin.de/~zhangg/pages/teaching/pages/d01.html',
        'http://people.f4.htw-berlin.de/~zhangg/pages/teaching/pages/d06.html',
        'http://people.f4.htw-berlin.de/~zhangg/pages/teaching/pages/d08.html'
    ]

    def parse(self, response):
         
        # - Extracting the needed attributes.
        title = response.xpath('//title/text()').extract()[0]
        text = ' '.join([t.strip() for t in response.xpath('//body/text()').extract()]).strip()
        hrefs =  [l.split('.')[0] for l in response.xpath('//a/@href').extract()]
                      
        # - Printing the extracted data for logging. 
        print("\n ----DATA\n\n%s\n" % json.dumps({title : {'text': text, 
                        'refs' : hrefs }}))
        
        # - Yielding the extracted dataset as dict
        yield {'title': title,
               'text': text, 
               'refs' : hrefs }
        # - Here we find the actual spider.
        # - This goes through all a_hrefs and scrape the needed data.
        for next_page in response.xpath('//a/@href').extract():
            if next_page is None:
                continue
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

Overwriting ./my_spider.py


### b) Execution

In [4]:
# - Here we call the scrapy process.
# 
# - This is needed to call this inside the notebook.
# - Scrapy concatenates all outputs (with destroying the json)
#   so we need to remove it first.

#   !!Attention!!   This will execute rm (!) on the the OS.  
#                   Won't work on windows.
!rm ./my_spider.json

# - Executing the spider.
pout = check_output('scrapy runspider my_spider.py -o my_spider.json', shell=True)
# - Writing the output into logfile.
with open('./my_spider.log', 'w') as logfile:
    logfile.write(pout.decode("utf-8"))

### Reading the crawled Data

In [5]:
# - Reading the Data from json into python obj-structure.-
with open('./my_spider.json') as infile:
    data = json.loads(''.join([line.strip() for line in infile.readlines()]))   
    
# - Removing all duplicates
data = list({d['title']:d for d in data}.values())

df = pd.DataFrame.from_dict(data)
df.index = df['title']
df

,refs,text,title
title,,,
d01,"[d02, d03, d04]",Given a character sequence and a defined docum...,d01
d06,[d07],"In text classification, we are given a descrip...",d06
d08,[],s is a spam page. tokens stopwords index posti...,d08
d04,"[d01, d02, d03, d05]",To gain the speed benefits of indexing at retr...,d04
d02,"[d03, d04, d01, d05]",Token normalization is the process of canonica...,d02
d03,"[d04, d01, d02, d05]","For English, an alternative to making every to...",d03
d07,[d06],Using a supervised learning method or learning...,d07
d05,[d04],Index the documents that each term occurs in b...,d05


## c)  Page Rank

$$
r_{k+1}(p_i) = d \cdot \left( \sum_{pj \in B_{p_i}}   \frac{r_k(pj)}{|pj|}  + \sum_{pj,\,|pj|=0}  \frac{r_k(p_j)}{N} \right) + \frac{t}{N}
$$



In [6]:
delta = 0.04 
N = len(data)
t = 0.05
d = 1-t

graph = [ {
    'node': d['title'], 
    'out':  d['refs'], 
    'in' : [n['title'] for n in data if d['title'] in n['refs']],
    'rank': 1.0 / N,          # (1 / n) \leq 1 \forall n \in \mathbb{N}
    'rank_': 1.0 + delta      # mocking rank_ 
} for d in data ]

pd.DataFrame.from_dict(graph)

,in,node,out,rank,rank_
0,"[d04, d02, d03]",d01,"[d02, d03, d04]",0.125,1.04
1,[d07],d06,[d07],0.125,1.04
2,[],d08,[],0.125,1.04
3,"[d01, d02, d03, d05]",d04,"[d01, d02, d03, d05]",0.125,1.04
4,"[d01, d04, d03]",d02,"[d03, d04, d01, d05]",0.125,1.04
5,"[d01, d04, d02]",d03,"[d04, d01, d02, d05]",0.125,1.04
6,[d06],d07,[d06],0.125,1.04
7,"[d04, d02, d03]",d05,[d04],0.125,1.04


In [7]:
"""Calculating the rank."""

# - Functions to calc the rank
abs_pj = lambda pj: [len(page['out']) for page in graph if page['node'] == pj][0]
get_rank = lambda pj: [p['rank'] for p in graph if p['node'] == pj][0]

_r = lambda pi: ( 
    d * (
        np.sum( [ get_rank(pj) / abs_pj(pj) for pj in pi['in'] ] ) + 
        np.sum( [ get_rank(pj['node']) / N for pj in graph if abs_pj(pj['node']) == 0 ])
    ) + (t / N) )

# - The iterations of the pageranks
while not np.sum([abs(g['rank'] - g['rank_']) for g in graph]) < delta :
    for node in graph:
        node['rank_'], node['rank'] = node['rank'], _r(node)

# - Put the calcuated data inside a DataFrame
df = pd.DataFrame(graph)
df.index = df['node']
# - Writing to file
df[['node', 'rank']].to_csv('rank.txt', index=False)
pprint(df['rank'])
print("+ ----------------")
pprint(np.sum(df['rank']))

node
d01    0.121327
d06    0.143281
d08    0.008755
d04    0.220873
d02    0.128147
d03    0.128602
d07    0.143407
d05    0.120725
Name: rank, dtype: float64
+ ----------------
1.0151158974735404


In [8]:
pd.DataFrame.from_dict(graph)

,in,node,out,rank,rank_
0,"[d04, d02, d03]",d01,"[d02, d03, d04]",0.121327,0.110156
1,[d07],d06,[d07],0.143281,0.139844
2,[],d08,[],0.008755,0.021094
3,"[d01, d02, d03, d05]",d04,"[d01, d02, d03, d05]",0.220873,0.221763
4,"[d01, d04, d03]",d02,"[d03, d04, d01, d05]",0.128147,0.125994
5,"[d01, d04, d02]",d03,"[d04, d01, d02, d05]",0.128602,0.126230
6,[d06],d07,[d06],0.143407,0.141606
7,"[d04, d02, d03]",d05,[d04],0.120725,0.121327


## d) Index

### Tokenizing

In [9]:
with open('./stop_words.txt') as infile:
    lines = infile.readlines()
stopwords = [word.strip("'").strip() 
             for line in lines for word 
             in line.split(', ') 
             if word.strip("'").strip() != '']

pattern = re.compile('[\W_]+')  

tokenizer = lambda d: [pattern.sub('', w.lower()) for w in d.split() if w not in stopwords]

docs = [{'title': d['title'], 'words': tokenizer(d['text'])} for d in data ]
#docs

### Indexing

In [10]:
words = [word for doc in docs for word in doc['words']]

iindx = pd.DataFrame.from_dict({ doc['title']: [ 
    doc['words'].count(word) for word in words ]
    for doc in docs })
iindx.index = words

iindx = iindx[~iindx.index.duplicated(keep='first')]
iindx.to_csv('index.txt')
iindx.head()

,d01,d02,d03,d04,d05,d06,d07,d08
given,1,0,0,0,0,1,0,0
character,1,1,0,0,0,0,0,0
sequence,1,0,0,0,0,0,0,0
defined,1,0,0,0,0,0,0,0
document,1,0,0,1,0,1,0,0


## Searching

###  e)  TF - IDF

In [11]:
idx = pd.read_csv('./index.txt', index_col=0)

def df(word, doc):
    f = int(idx[doc].loc[word])   
    return 1+log(f) if f > 0 else 0
    
def idf(word):
    N = idx.shape[0]
    n_i = np.sum(idx.loc[word].map(lambda x: 0 if x == 0 else 1))
    
    return log (N / n_i)

tfidf = lambda w, d: (df(w, d) * idf(w))
weights = lambda w : [{d: tfidf(w,d)} for d in idx.columns.values.tolist()]
pd.DataFrame.from_dict([{word: weights(word) for word in words}])

tfiidx = pd.DataFrame.from_dict({ doc['title']: [ 
    tfidf(word, doc['title']) for word in words ]
    for doc in docs })
tfiidx.index = words
tfiidx = tfiidx[~tfiidx.index.duplicated(keep='first')]

In [12]:
def search (words):
    assert type(words) == list
    
    return pd.DataFrame([tfiidx.loc[w] for w in words]).product()
    
s_terms = [['tokens'], ['index'], ['classification'], ['tokens', 'classification']]

search_df = pd.DataFrame([search(t) for t in s_terms])
search_df.index=[' AND '.join(s) for s in s_terms]

In [13]:
search_df

,d01,d02,d03,d04,d05,d06,d07,d08
tokens,3.091042,5.23359,3.091042,5.233590,0.000000,0.000000,0.000000,7.376137
index,0.000000,0.00000,0.000000,3.601868,6.098493,0.000000,0.000000,8.595117
classification,0.000000,0.00000,0.000000,0.000000,0.000000,3.601868,3.601868,8.595117
tokens AND classification,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,63.398766


In [14]:
search_df.to_csv('tfidf_search.txt')

## f) Scoring tfidf und pagerank

$$\mbox{score} = \mbox{tf_idf}(w, d) \cdot \mbox{rank}(d)$$

In [15]:
r = pd.read_csv('rank.txt', index_col=0)
s = pd.read_csv('tfidf_search.txt', index_col=0)

score = pd.DataFrame(r.values * s.T.values, index=r.index, columns=s.index).T
score.to_csv('pageranke_search.txt')
score

node,d01,d06,d08,d04,d02,d03,d07,d05
tokens,0.375026,0.749874,0.027062,1.155960,0.000000,0.000000,0.000000,0.890483
index,0.000000,0.000000,0.000000,0.795556,0.781502,0.000000,0.000000,1.037644
classification,0.000000,0.000000,0.000000,0.000000,0.000000,0.463207,0.516532,1.037644
tokens AND classification,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.653806
